## AGRICULTURAL PRICE PREDICTION - PRODUCTION MODEL
============================================================

This notebook trains a machine learning model to predict retail prices
of agricultural commodities in Kenya using the Kamis dataset.

#### Author: Kyalo Josephine Kathini

##### Date: 2025

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

 ### KAMIS AGRICULTURAL PRICE PREDICTION MODE

 #### 1. Data loading and cleaning

In [14]:
 def clean_price_column(col):
    """
    Robust price cleaning for various formats:
    - Removes '/Kg', '/kg', 'KES', 'Ksh'
    - Handles missing values ('-', 'NA', etc.)
    - Converts to numeric
    """
    if col.dtype == 'object':
        col = col.astype(str)
        col = col.replace(['-', '', 'NA', 'N/A', 'nan', 'NaN', 'None'], np.nan)
        col = col.str.replace('/kg', '', case=False, regex=False)
        col = col.str.replace('/Kg', '', case=False, regex=False)
        col = col.str.replace('ksh', '', case=False, regex=False)
        col = col.str.replace('kes', '', case=False, regex=False)
        col = col.str.replace(' ', '', regex=False)
        col = col.str.replace(',', '', regex=False)
        col = pd.to_numeric(col, errors='coerce')
    return col

In [15]:
# Load data
print("\nLoading data...")
df = pd.read_csv("../data/raw/kamis_data.csv")
print(f"Loaded {len(df):,} rows and {len(df.columns)} columns")


Loading data...
Loaded 310,304 rows and 12 columns


In [16]:
# Convert date  
df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
df = df.dropna(subset=["Date"])
df = df.sort_values("Date").reset_index(drop=True)
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}") 

Date range: 2008-04-01 00:00:00 to 2025-09-17 00:00:00


In [17]:
# Clean price columns
df['Retail'] = clean_price_column(df['Retail'])
df['Wholesale'] = clean_price_column(df['Wholesale'])
df['Supply Volume'] = pd.to_numeric(df['Supply Volume'], errors='coerce')
print(f"✓ Cleaned price columns")

✓ Cleaned price columns


In [18]:
initial_count = len(df)
df = df.dropna(subset=['Retail'])
print(f"Removed {initial_count - len(df):,} rows without retail price")

Removed 53,690 rows without retail price


In [19]:
# Remove unrealistic outliers EARLY
print(f"\nPrice statistics before outlier removal:")
print(f"  Min: {df['Retail'].min():.2f}, Max: {df['Retail'].max():.2f}")
print(f"  Mean: {df['Retail'].mean():.2f}, Median: {df['Retail'].median():.2f}")

df = df[(df['Retail'] >= 1) & (df['Retail'] <= 5000)]
print(f"\nPrice statistics after outlier removal (1-5000 KES):")
print(f"  Min: {df['Retail'].min():.2f}, Max: {df['Retail'].max():.2f}")
print(f"  Mean: {df['Retail'].mean():.2f}, Median: {df['Retail'].median():.2f}")
print(f"Final dataset: {len(df):,} rows")


Price statistics before outlier removal:
  Min: 0.01, Max: 100000.00
  Mean: 170.71, Median: 100.00

Price statistics after outlier removal (1-5000 KES):
  Min: 1.00, Max: 5000.00
  Mean: 164.35, Median: 100.00
Final dataset: 256,167 rows


 #### 2. Feature Engineering

In [20]:
# Sorting data properly for time series
df = df.sort_values(['Commodity', 'Market', 'County', 'Date']).reset_index(drop=True)
print("Sorted data by Commodity → Market → County → Date")

Sorted data by Commodity → Market → County → Date


#### 2.1 TEMPORAL FEATURES

In [21]:
print("\n[2.1] Creating temporal features...")
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['week_of_year'] = df['Date'].dt.isocalendar().week
df['day_of_year'] = df['Date'].dt.dayofyear
df['quarter'] = df['Date'].dt.quarter
df['day_of_week'] = df['Date'].dt.dayofweek
df['day_of_month'] = df['Date'].dt.day


[2.1] Creating temporal features...


In [22]:
# Cyclical encoding  
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['week_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
df['week_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)

In [23]:
# Kenya-specific agricultural seasons
df['is_long_rains'] = df['month'].isin([3, 4, 5]).astype(int)
df['is_short_rains'] = df['month'].isin([10, 11, 12]).astype(int)
df['is_planting_season'] = df['month'].isin([3, 4, 10]).astype(int)
df['is_harvest_season'] = df['month'].isin([7, 8, 1, 2]).astype(int)

print(f"  Created 18 temporal features")

  Created 18 temporal features


#### 2.2 LAG FEATURES

In [24]:
print("\n[2.2] Creating lag features...")
group_cols = ['Commodity', 'Market', 'County']


[2.2] Creating lag features...


In [25]:
# Retail price lags
lags = [1, 2, 3, 7, 14, 30]
for lag in lags:
    df[f'retail_lag_{lag}'] = df.groupby(group_cols)['Retail'].shift(lag)
print(f"  ✓ Created {len(lags)} retail lag features")

  ✓ Created 6 retail lag features


In [26]:
# Wholesale price lags  
if 'Wholesale' in df.columns:
    for lag in [1, 2, 7]:
        df[f'wholesale_lag_{lag}'] = df.groupby(group_cols)['Wholesale'].shift(lag)
    print(f"  Created 3 wholesale lag features")

  Created 3 wholesale lag features


In [27]:
# Supply volume lags
if 'Supply Volume' in df.columns:
    df['Supply Volume'] = df['Supply Volume'].fillna(0)
    for lag in [1, 7, 14]:
        df[f'supply_lag_{lag}'] = df.groupby(group_cols)['Supply Volume'].shift(lag)
    print(f"  ✓ Created 3 supply lag features")

  ✓ Created 3 supply lag features


#### 2.3 ROLLING WINDOW FEATURES (Trends and Volatility)

In [28]:
print("\n[2.3] Creating rolling window features...")
windows = [7, 14, 30]


[2.3] Creating rolling window features...


In [31]:
original_index = df.index
df_reset = df.reset_index(drop=True)

for window in windows:
    print(f"  Creating features for {window}-day window...")
    def calculate_rolling_stats(group, window):
        group = group.sort_values('Date')
        group[f'retail_ma_{window}'] = group['Retail'].shift(1).rolling(window=window, min_periods=1).mean()
        group[f'retail_std_{window}'] = group['Retail'].shift(1).rolling(window=window, min_periods=1).std()
        group[f'retail_min_{window}'] = group['Retail'].shift(1).rolling(window=window, min_periods=1).min()
        group[f'retail_max_{window}'] = group['Retail'].shift(1).rolling(window=window, min_periods=1).max()
        return group
  
    df_grouped = df_reset.groupby(group_cols).apply(
        lambda x: calculate_rolling_stats(x, window)
    )
    
    
    df_grouped = df_grouped.reset_index(drop=True)
    
     
    for col in [f'retail_ma_{window}', f'retail_std_{window}', f'retail_min_{window}', f'retail_max_{window}']:
        df[col] = df_grouped[col]  

print(f"  Created {len(windows) * 4} rolling window features")

  Creating features for 7-day window...
  Creating features for 14-day window...
  Creating features for 30-day window...
  Created 12 rolling window features


#### 2.4  MOMENTUM FEATURES (Price trends)

In [35]:
print("\n[2.4] Creating momentum features...")
df['price_volatility_7d'] = df['retail_max_7'] - df['retail_min_7']
df['price_volatility_30d'] = df['retail_max_30'] - df['retail_min_30']
 
df['price_momentum_7d'] = np.nan
df['price_momentum_30d'] = np.nan
df['price_change_rate'] = np.nan

 
for name, group in df.groupby(group_cols):
    group_idx = group.index
    group_sorted = group.sort_values('Date')
    
    df.loc[group_idx, 'price_momentum_7d'] = group_sorted['Retail'].diff(7).values
    df.loc[group_idx, 'price_momentum_30d'] = group_sorted['Retail'].diff(30).values
    df.loc[group_idx, 'price_change_rate'] = group_sorted['Retail'].pct_change().values

print(f"  Created 5 momentum features")


[2.4] Creating momentum features...
  Created 5 momentum features


#### 2.5   AGGREGATE FEATURES  

In [36]:
print("\n[2.5] Creating aggregate features...")

# Initialize new columns
df['commodity_expanding_mean'] = np.nan
df['county_expanding_mean'] = np.nan
df['market_expanding_mean'] = np.nan


[2.5] Creating aggregate features...


In [37]:
for name, group in df.groupby('Commodity'):
    group_idx = group.index
    group_sorted = group.sort_values('Date')
    df.loc[group_idx, 'commodity_expanding_mean'] = group_sorted['Retail'].shift(1).expanding(min_periods=1).mean().values

for name, group in df.groupby('County'):
    group_idx = group.index
    group_sorted = group.sort_values('Date')
    df.loc[group_idx, 'county_expanding_mean'] = group_sorted['Retail'].shift(1).expanding(min_periods=1).mean().values

for name, group in df.groupby('Market'):
    group_idx = group.index
    group_sorted = group.sort_values('Date')
    df.loc[group_idx, 'market_expanding_mean'] = group_sorted['Retail'].shift(1).expanding(min_periods=1).mean().values

In [38]:
# Price deviations from averages
df['price_vs_commodity_avg'] = df['Retail'] - df['commodity_expanding_mean']
df['price_vs_county_avg'] = df['Retail'] - df['county_expanding_mean']
df['price_vs_market_avg'] = df['Retail'] - df['market_expanding_mean']

In [39]:
# Supply features
if 'Supply Volume' in df.columns:
    df['supply_volume_log'] = np.log1p(df['Supply Volume'])
    df['supply_change'] = np.nan
    for name, group in df.groupby(group_cols):
        group_idx = group.index
        group_sorted = group.sort_values('Date')
        df.loc[group_idx, 'supply_change'] = group_sorted['Supply Volume'].diff().values

print(f"  ✓ Created 8 aggregate features")

  ✓ Created 8 aggregate features


### 3. Categorical Encoding

In [40]:
# Get top categories to reduce dimensionality
print("\nSelecting top categories:")
for col in ['Commodity', 'County', 'Market']:
    if col in df.columns:
        top_categories = df[col].value_counts().head(10).index
        print(f"  {col}: {len(top_categories)} categories")
        
        for category in top_categories:
            safe_name = category.replace(' ', '_').replace('/', '_').replace('-', '_')
            feature_name = f"{col}_{safe_name}"
            df[feature_name] = (df[col] == category).astype(int)

print("Created categorical dummy variables")


Selecting top categories:
  Commodity: 10 categories
  County: 10 categories
  Market: 10 categories
Created categorical dummy variables


### 4. Prepare Training Data

In [41]:
# Define feature columns 
exclude_cols = [
    'Date', 'Retail', 'Wholesale',   
    'Commodity', 'County', 'Market',   
    'Classification', 'Grade', 'Sex', 'ProductID', 'ProductName', 'Supply Volume'
]

feature_cols = [col for col in df.columns if col not in exclude_cols]

In [42]:
# Remove rows with NaN in critical features
critical_features = ['retail_lag_1', 'retail_lag_2', 'month', 'year']
df_clean = df.dropna(subset=critical_features + ['Retail'])
print(f"Removed rows with missing critical features")
print(f"Remaining rows: {len(df_clean):,}")

Removed rows with missing critical features
Remaining rows: 240,061


In [43]:
# Fill remaining NaN with median  
for col in feature_cols:
    if col in df_clean.columns and df_clean[col].isna().any():
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

In [44]:
# Final feature matrix
X = df_clean[feature_cols].copy()
y = df_clean['Retail'].copy()

In [45]:
# Handle any infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

In [46]:
print(f"\nFINAL DATASET READY:")
print(f"  Total features: {X.shape[1]}")
print(f"  Total samples: {X.shape[0]:,}")
print(f"  Target (Retail) - Mean: {y.mean():.2f} KES, Std: {y.std():.2f} KES")
print(f"  Date range: {df_clean['Date'].min()} to {df_clean['Date'].max()}")
print(f"  NaN values: {X.isna().sum().sum()}")


FINAL DATASET READY:
  Total features: 82
  Total samples: 240,061
  Target (Retail) - Mean: 164.52 KES, Std: 198.62 KES
  Date range: 2021-05-24 00:00:00 to 2025-09-17 00:00:00
  NaN values: 0


### 5. Model Training and Evaluation

In [47]:
# Feature scaling
scaler = StandardScaler()

In [48]:
# Time Series Split 
tscv = TimeSeriesSplit(n_splits=5)

In [49]:
# Define models
models = {
    'Random Forest': RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=50,
        min_samples_leaf=20,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1,
        verbose=0
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        min_samples_split=50,
        random_state=42,
        verbose=0
    )
}

results = {}
best_model_obj = None
best_model_name = None

In [50]:
for model_name, model in models.items():
    print(f"\n{'='*80}")
    print(f"Training: {model_name}")
    print(f"{'='*80}")
    
    fold_mae, fold_rmse, fold_r2, fold_mape = [], [], [], []
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
        # Split data
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
        
        # Scale features
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train model
        model.fit(X_train_scaled, y_train)
        
        # Predict
        y_pred = model.predict(X_test_scaled)
        
        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred) * 100
        
        fold_mae.append(mae)
        fold_rmse.append(rmse)
        fold_r2.append(r2)
        fold_mape.append(mape)
        
        print(f"Fold {fold + 1}: MAE={mae:.2f} KES, RMSE={rmse:.2f} KES, R²={r2:.4f}, MAPE={mape:.2f}%")
    
    # Store results
    results[model_name] = {
        'MAE': (np.mean(fold_mae), np.std(fold_mae)),
        'RMSE': (np.mean(fold_rmse), np.std(fold_rmse)),
        'R²': (np.mean(fold_r2), np.std(fold_r2)),
        'MAPE': (np.mean(fold_mape), np.std(fold_mape))
    }
    
    print(f"\n{model_name} - AVERAGE RESULTS:")
    print(f"  MAE:  {np.mean(fold_mae):.2f} ± {np.std(fold_mae):.2f} KES")
    print(f"  RMSE: {np.mean(fold_rmse):.2f} ± {np.std(fold_rmse):.2f} KES")
    print(f"  R²:   {np.mean(fold_r2):.4f} ± {np.std(fold_r2):.4f}")
    print(f"  MAPE: {np.mean(fold_mape):.2f} ± {np.std(fold_mape):.2f}%")



Training: Random Forest
Fold 1: MAE=14.20 KES, RMSE=37.76 KES, R²=0.8598, MAPE=24.09%
Fold 2: MAE=24.79 KES, RMSE=144.54 KES, R²=0.6784, MAPE=18.41%
Fold 3: MAE=17.61 KES, RMSE=43.55 KES, R²=0.9690, MAPE=23.92%
Fold 4: MAE=11.90 KES, RMSE=61.93 KES, R²=0.9212, MAPE=13.64%
Fold 5: MAE=9.01 KES, RMSE=27.31 KES, R²=0.9746, MAPE=10.85%

Random Forest - AVERAGE RESULTS:
  MAE:  15.50 ± 5.43 KES
  RMSE: 63.02 ± 42.28 KES
  R²:   0.8806 ± 0.1092
  MAPE: 18.18 ± 5.34%

Training: Gradient Boosting
Fold 1: MAE=6.26 KES, RMSE=21.23 KES, R²=0.9557, MAPE=12.02%
Fold 2: MAE=12.71 KES, RMSE=56.50 KES, R²=0.9509, MAPE=6.98%
Fold 3: MAE=9.11 KES, RMSE=17.13 KES, R²=0.9952, MAPE=7.57%
Fold 4: MAE=4.82 KES, RMSE=12.10 KES, R²=0.9970, MAPE=4.39%
Fold 5: MAE=4.14 KES, RMSE=9.16 KES, R²=0.9971, MAPE=3.94%

Gradient Boosting - AVERAGE RESULTS:
  MAE:  7.41 ± 3.15 KES
  RMSE: 23.22 ± 17.15 KES
  R²:   0.9792 ± 0.0212
  MAPE: 6.98 ± 2.89%


### 6. Feature Importance Analysis

In [51]:
# Training Random Forest on all data for feature importance
rf_final = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=50,
    random_state=42,
    n_jobs=-1
)

X_scaled = scaler.fit_transform(X)
rf_final.fit(X_scaled, y)

RandomForestRegressor(max_depth=15, min_samples_split=50, n_jobs=-1,
                      random_state=42)

In [52]:
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_final.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print(feature_importance.head(20).to_string(index=False))


Top 20 Most Important Features:
                 feature  importance
     price_vs_county_avg    0.828165
            retail_lag_1    0.118420
            retail_lag_2    0.028121
   county_expanding_mean    0.010293
       price_change_rate    0.009765
  price_vs_commodity_avg    0.002888
            retail_lag_3    0.000748
     price_vs_market_avg    0.000653
commodity_expanding_mean    0.000416
   market_expanding_mean    0.000106
   Commodity_Maize_Flour    0.000084
           retail_lag_30    0.000038
           retail_lag_14    0.000033
   Commodity_Wheat_Flour    0.000031
            retail_lag_7    0.000029
       price_momentum_7d    0.000016
         wholesale_lag_2    0.000016
         wholesale_lag_1    0.000012
             Market_Aram    0.000011
            retail_ma_30    0.000010


### 7. Final results and recommendations

In [53]:
best_model_name = min(results.keys(), key=lambda x: results[x]['MAE'][0])
best_mae = results[best_model_name]['MAE'][0]
best_rmse = results[best_model_name]['RMSE'][0]
best_r2 = results[best_model_name]['R²'][0]
best_mape = results[best_model_name]['MAPE'][0]

In [54]:
print(f"\nBEST MODEL: {best_model_name}")
print(f"{'='*80}")
print(f"  MAE:  {best_mae:.2f} KES")
print(f"  RMSE: {best_rmse:.2f} KES")
print(f"  R²:   {best_r2:.4f}")
print(f"  MAPE: {best_mape:.2f}%")
print(f"\n  MAE as % of mean price: {(best_mae / y.mean() * 100):.1f}%")


BEST MODEL: Gradient Boosting
  MAE:  7.41 KES
  RMSE: 23.22 KES
  R²:   0.9792
  MAPE: 6.98%

  MAE as % of mean price: 4.5%


In [55]:
print(f"\nMODEL COMPARISON:")
print(f"{'='*80}")
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    print(f"  MAE:  {metrics['MAE'][0]:.2f} ± {metrics['MAE'][1]:.2f} KES")
    print(f"  RMSE: {metrics['RMSE'][0]:.2f} ± {metrics['RMSE'][1]:.2f} KES")
    print(f"  R²:   {metrics['R²'][0]:.4f} ± {metrics['R²'][1]:.4f}")


MODEL COMPARISON:

Random Forest:
  MAE:  15.50 ± 5.43 KES
  RMSE: 63.02 ± 42.28 KES
  R²:   0.8806 ± 0.1092

Gradient Boosting:
  MAE:  7.41 ± 3.15 KES
  RMSE: 23.22 ± 17.15 KES
  R²:   0.9792 ± 0.0212


### 8. Saving the model

In [56]:
import joblib
joblib.dump(rf_final, '../models/price_model.pkl')
joblib.dump(scaler, '../models/scaler.pkl')
print("Model saved to ../models/")

Model saved to ../models/
